In [522]:
import numpy as np
import random
from scipy.special import softmax

In [523]:
values = np.zeros((3**9, ))
# compute a value for each of the states of the board. 
# This is kind of like using a minimax tree with depth 1, 
# for which the desirability of a future state is not computed explicitly depending upon actual wins and losses 
# but rather learned from past experience
gamma = 0.9
# discount factor: states that are close to the end of the game are rewarded/punished 
# more than states that are towards the beginning of the game

In [524]:
def serialize(board):
    # hash function for boards, converts board to int so I can compute value of states
    serial = 0
    for i in range(9):
        serial = serial + board[i]*3**(i)
    return int(serial)

In [525]:
def print_board(serial):
    # print board from seial number
    board = np.zeros((9,))
    for i in range(9):
        board[i] = serial%3
        serial = serial/3
    print(board[0:3])
    print(board[3:6])
    print(board[6:9])

In [526]:
def rademacher(x):
    # utility function, maps:
    # even -> 1
    # odd -> -1
    return 1-2*(x%2)

In [527]:
class game:
    def __init__(self, training, generation=0):
        self.board = np.zeros((9,))
        # self.board is a 1D array representing the board state.
        # 0 is empty, 1 is X, 2 is O

        self.num_moves = 0
        
        self.history = []
        # list of board states that are played during the game
        
        self.training = training
        # is the game a 
        # training == 1: "training" game (values are being learned) or an 
        # training == 0: "actual" game (no learning; just play the best move)
        
        self.player = int(np.random.uniform()>0.5)
        # if a training game, which player should play intelligently
        
        self.generation = generation
        # generation number for value convergence. 
        # The weights I use (1/log(generation)) don't actually lead to convergence 
        # L2 norm of series should be finite for convergence
        # but whatever
        
        
    def clear_history(self):
        # not used
        self.board = np.zeros((9,))
        self.history = []
        self.num_moves = 0
        
    def make_move(self, move):
        self.board[move] = self.num_moves%2+1

        self.history += [serialize(self.board)]
        # append board state to history, so that the value of the state can be updated at the end of the game
        
        self.num_moves = self.num_moves+1
        
    def game_won(self):
        if (self.board[0] == self.board[1] and self.board[0] == self.board[2] and self.board[0] != 0) or \
            (self.board[0] == self.board[4] and self.board[0] == self.board[8] and self.board[0] != 0) or \
            (self.board[0] == self.board[3] and self.board[0] == self.board[6] and self.board[0] != 0) or \
            (self.board[1] == self.board[4] and self.board[1] == self.board[7] and self.board[1] != 0) or \
            (self.board[2] == self.board[4] and self.board[2] == self.board[6] and self.board[2] != 0) or \
            (self.board[2] == self.board[5] and self.board[2] == self.board[8] and self.board[2] != 0) or \
            (self.board[3] == self.board[4] and self.board[3] == self.board[5] and self.board[3] != 0) or \
            (self.board[6] == self.board[7] and self.board[6] == self.board[8] and self.board[6] != 0):
            return True
        return False
    
    def legal_moves(self):
        # returns a list of legal moves
        moves = []
        for i in range(9):
            if self.board[i] == 0:
                moves += [i]
        return moves
    
    def get_move(self):
        moves = self.legal_moves()
        vs = np.zeros((len(moves),))
        potential_boards = [self.board.copy() for i in range(len(moves))]
        # list of potential boards, one for each legal move

        for ind, m in enumerate(moves):
            potential_boards[ind][m] = self.num_moves%2+1
            vs[ind] = values[serialize(potential_boards[ind])]
            # get the values of these boards. 
            # Value of a state indicates the goodness of leaving the board in that state after your turn, 
            # so you want to play in such a way as to maximize the value of the state of the potential board   
            
        if self.training:
            # if training, play randomly on every other move
            if self.player == self.num_moves%2:
                return moves[np.argmax(np.random.multinomial(1, softmax(vs)))]
                # use boltzmann style distribution to compute next move 
                # plays less optimal moves with some probability to encourage exploration
            else:
                return random.sample(moves, 1)   
        else:
            for ind, m in enumerate(moves):
                print(potential_boards[ind][0:3])
                print(potential_boards[ind][3:6])
                print(potential_boards[ind][6:9])
                print(vs[ind])
            return moves[np.argmax(vs)]
            # if actual game, just play the best move
            
    
    def update_values(self):
        winner = 2-(self.num_moves%2) # 1 if p1 won, 2 if p2 won
        for ind, b in enumerate(list(reversed(self.history))):
            values[b] = values[b]+rademacher(ind)*gamma**ind/np.log(self.generation+1)
            # increases value if it leads to "game over" after an even number of moves, 
            # decreases value if it leads to "game over" after an off number of moves
            
    def print_game(self):
        print(self.board[0:3])
        print(self.board[3:6])
        print(self.board[6:9])


In [528]:
def play_game(generation):
    training_game = game(1, generation)
    while training_game.num_moves < 9 and not training_game.game_won():
        training_game.make_move(training_game.get_move())
    #training_game.print_game()
    if training_game.game_won():
        training_game.update_values()
        #print(serialize(training_game.board))
    #training_game.clear_history()

In [529]:
for i in range(1, 100000):
    play_game(i)
#     if winner == 1:
#         print('p1 won')
#     elif winner == 2:
#         print('p2 won')
#     else:
#         print('draw')

In [530]:
v = 13325
print_board(v)
print(values[v])

[2. 1. 1.]
[1. 2. 0.]
[0. 0. 2.]
50.000690490708465


In [531]:
role = input("0 to play first, 1 to play second: ")
bot_turn = 1-role
actual_game = game(0)
turn = 0
while actual_game.num_moves < 9 and not actual_game.game_won():
    if not turn == bot_turn:
        move = input("play_move: ")
        actual_game.make_move(move)
    else:
        actual_game.make_move(actual_game.get_move())
    actual_game.print_game()
    turn = 1-turn

if not actual_game.game_won() and actual_game.num_moves == 9:
    print('tie')
if actual_game.game_won() and actual_game.num_moves%2 == bot_turn-1:
    print('you lost')
if actual_game.game_won() and actual_game.num_moves%2 == bot_turn:
    print('you won')
    

0 to play first, 1 to play second: 1
[1. 0. 0.]
[0. 0. 0.]
[0. 0. 0.]
-220.97841597482147
[0. 1. 0.]
[0. 0. 0.]
[0. 0. 0.]
-227.83757416467014
[0. 0. 1.]
[0. 0. 0.]
[0. 0. 0.]
-212.2125616164353
[0. 0. 0.]
[1. 0. 0.]
[0. 0. 0.]
-233.00841083086186
[0. 0. 0.]
[0. 1. 0.]
[0. 0. 0.]
-89.57149665940086
[0. 0. 0.]
[0. 0. 1.]
[0. 0. 0.]
-232.5393812592532
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
2519.6928912136386
[0. 0. 0.]
[0. 0. 0.]
[0. 1. 0.]
-230.35630710484548
[0. 0. 0.]
[0. 0. 0.]
[0. 0. 1.]
-209.78100593972758
[0. 0. 0.]
[0. 0. 0.]
[1. 0. 0.]
play_move: 4
[0. 0. 0.]
[0. 2. 0.]
[1. 0. 0.]
[1. 0. 0.]
[0. 2. 0.]
[1. 0. 0.]
-83.5332795790565
[0. 1. 0.]
[0. 2. 0.]
[1. 0. 0.]
-77.33220086188193
[0. 0. 1.]
[0. 2. 0.]
[1. 0. 0.]
-90.26478496354996
[0. 0. 0.]
[1. 2. 0.]
[1. 0. 0.]
-88.54748303361816
[0. 0. 0.]
[0. 2. 1.]
[1. 0. 0.]
-86.99105987728035
[0. 0. 0.]
[0. 2. 0.]
[1. 1. 0.]
328.18158731842766
[0. 0. 0.]
[0. 2. 0.]
[1. 0. 1.]
-77.91686620164704
[0. 0. 0.]
[0. 2. 0.]
[1. 1. 0.]
play_move: 8
[0